In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
#apariciones primeros 3 dias
appearances_train = pd.read_csv("appearances_first_3_days.csv", header=None, names=["device_id", "date", "seconds_until_next_appearence"])

In [3]:
#apariciones segundos 3 dias
appearances_test =  pd.read_csv("appearances_second_3_days.csv", header=None, names=["device_id", "date", "seconds_until_next_appearence"])

In [4]:
#convertions_train = pd.read_csv("auctions_deltas_to_convertion_train.csv",header=None, names=["device_id", "date", "seconds_until_convertion"])

In [5]:
#convertions_test = pd.read_csv("auctions_deltas_to_convertion_test.csv",header=None, names=["device_id", "date", "seconds_until_convertion"])

In [6]:
#ids a predecir en ambos problemas
ids_to_predict = pd.read_csv("target_competencia_ids.csv")

In [7]:
ids_to_predict = ids_to_predict[["ref_hash"]]

In [8]:
appearances_train["date"] = pd.to_datetime(appearances_train["date"])

In [9]:
appearances_test["date"] = pd.to_datetime(appearances_test["date"])

In [10]:
#convertions_test["date"] = pd.to_datetime(convertions_test["date"])

In [11]:
#convertions_train["date"] = pd.to_datetime(convertions_train["date"])

In [12]:
events = pd.read_csv("events.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (13,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
#appearances_test["is_weekend"] = appearances_test.apply(lambda row: row["date"].weekday()>4 , axis=1)

In [14]:
#convertions_test["is_weekend"] = convertions_test.apply(lambda row: row["date"].weekday()>4 , axis=1)

In [15]:
#convertions_train["is_weekend"] = convertions_train.apply(lambda row: row["date"].weekday()>4 , axis=1)

In [16]:
ids_to_predict_numbers = ids_to_predict.copy()

In [17]:
#paso refhash a int sacandole los _sc o _st
ids_to_predict_numbers["ref_hash"] = ids_to_predict_numbers["ref_hash"].apply(lambda string: int(string.split("_")[0]))

In [18]:
#borro duplicados
ids_to_predict_numbers = ids_to_predict_numbers.drop_duplicates()

In [19]:
#por enunciado la fecha de los ids a predecir tiene que ser esta LA MISMA PARA TODOS
#o sea hay que predecir los segundos a partir de esa fecha
ids_to_predict_numbers["date"] = datetime(2019, 5, 27, 0,0,0)

In [20]:
ids_to_predict_numbers.head()

,ref_hash,date
0,1000169251625791246,2019-05-27
2,1000395625957344683,2019-05-27
4,1003027494996471685,2019-05-27
6,1006670001679961544,2019-05-27
8,1007573308966476713,2019-05-27


In [21]:
#la copio en 2 dfs distintos para usar, uno para predecir el tiempo en reaparecer en auction
#y otro para predecir el tiempo en convertir/instalar

In [22]:
ids_to_predict_numbers_appearances = ids_to_predict_numbers.copy()

In [23]:
ids_to_predict_numbers_convertion = ids_to_predict_numbers.copy()

In [24]:
appearances_train.head()

,device_id,date,seconds_until_next_appearence
0,4172466725848941608,2019-04-18 06:27:50.996158,848.908411
1,4172466725848941608,2019-04-18 06:41:59.904569,28513.038578
2,4172466725848941608,2019-04-18 14:37:12.943147,199.094384
3,4172466725848941608,2019-04-18 14:40:32.037531,185.775504
4,4172466725848941608,2019-04-18 14:43:37.813035,255.313251


In [25]:
ids_to_predict_numbers_appearances.head()

,ref_hash,date
0,1000169251625791246,2019-05-27
2,1000395625957344683,2019-05-27
4,1003027494996471685,2019-05-27
6,1006670001679961544,2019-05-27
8,1007573308966476713,2019-05-27


In [26]:
#renombro columna para poder mergear por id
ids_to_predict_numbers_appearances = ids_to_predict_numbers_appearances.rename(columns={"ref_hash": "device_id"})

In [27]:
ids_to_predict_numbers_appearances.head()

,device_id,date
0,1000169251625791246,2019-05-27
2,1000395625957344683,2019-05-27
4,1003027494996471685,2019-05-27
6,1006670001679961544,2019-05-27
8,1007573308966476713,2019-05-27


In [28]:
#les agrego los features

In [29]:
#appearances_train["is_weekend"] = appearances_train["data"].apply(lambda date: date.weekday()>4)

In [30]:
#ids_to_predict_numbers_appearances["is_weekend"] = ids_to_predict_numbers_appearances["date"].apply(lambda date: date["date"].weekday()>4)

In [31]:
#voy a agregar feature de APP mas usada para cada ID
most_used_app = events.groupby("ref_hash")["application_id"].agg(lambda x: x.mode()).rename_axis("device_id").to_frame().reset_index()


In [32]:
#arreglo que algunos casos la app id mas usada es una lista. porque la funcion .mode() devolvio mas de un elemento
most_used_app["application_id"] = most_used_app["application_id"].apply(lambda x: x[0] if type(x) == np.ndarray else x)

In [33]:
#SE LO AGREGO A LOS 3 DFS , cosa que voy a hacer con todos los nuevos features
#porque train (primeros 3 dias) es para entrenar, test (segundos 3 dias) es para testear, y competition es
#para submitear 

In [34]:
df_train = appearances_train.merge(most_used_app, on="device_id", how="left")

In [35]:
df_test = appearances_test.merge(most_used_app, on="device_id", how="left")
df_competition = ids_to_predict_numbers_appearances.merge(most_used_app, on="device_id", how="left")

In [36]:
#renombro para dejar mas claro que el nombre del nuevo feature
df_train = df_train.rename({"application_id": "most_used_app"}, axis="columns")
df_test = df_test.rename({"application_id": "most_used_app"}, axis="columns")
df_competition = df_competition.rename({"application_id": "most_used_app"}, axis="columns")

In [37]:
#reemplazo los nan de most used app por un nuemero no existente
df_train["most_used_app"] = df_train["most_used_app"].fillna(-1)
df_test["most_used_app"] = df_test["most_used_app"].fillna(-1)
df_competition["most_used_app"] = df_competition["most_used_app"].fillna(-1)

In [38]:
def is_android_or_macos(row):
    if row["user_agent"] is np.nan:
        return None
    agent = row["user_agent"].lower()
    if "android" in agent:
        return "android"
    elif "darwin" in agent:
        return "mac os"
    else:
        return "other"

In [39]:
#AHORA USO installs para crear feature de sistema operativo

In [40]:
installs = pd.read_csv("installs.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (4,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [41]:
installs["os"] = installs.apply(is_android_or_macos, axis=1)

In [42]:
os = installs[["ref_hash", "os"]].dropna()

In [43]:
#saco ids duplicados
os = os.drop_duplicates(subset="ref_hash", keep='first').rename({"ref_hash": "device_id"},axis="columns")

In [44]:
#lo agrego a DF, siempre left join
df_train = df_train.merge(os,how="left", on="device_id")
df_test = df_test.merge(os,how="left", on="device_id")
df_competition = df_competition.merge(os,how="left", on="device_id")

In [45]:
df_train["os"] = df_train["os"].fillna("other")
df_test["os"] = df_test["os"].fillna("other")
df_competition["os"] = df_competition["os"].fillna("other")

In [46]:
#PRUEBO OTRA VEZ con random forest
#antes tengo que convertir a numeros los campos que son string (en este caso solo OS)
#USO LABEL ENCODER
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [47]:
#encodeo columna os porque no puede ser string. tiene que ser numero
le.fit(df_train["os"])
df_train["os"] = le.transform(df_train["os"])
df_test["os"] = le.transform(df_test["os"])
df_competition["os"] = le.transform(df_competition["os"])

In [48]:
#ahora en vez de string tengo sus codigos
df_train["os"].head()

0    2
1    2
2    2
3    2
4    2
Name: os, dtype: int32

In [49]:
#Agrego features de cantidades de installs, auctions y events

In [50]:
#Se agregan los csv de features
df_eventos_feature = pd.read_csv("df_eventos_feature.csv")
auctions_installs_cantidades = pd.read_csv("auctions_installs_cantidades.csv")
features_cantidades = auctions_installs_cantidades.merge(df_eventos_feature, how = "outer", on = "device_id")

In [51]:
#lo agrego a DF, siempre left join
df_train = df_train.merge(features_cantidades,how="left", on="device_id")
df_test = df_test.merge(features_cantidades,how="left", on="device_id")
df_competition = df_competition.merge(features_cantidades,how="left", on="device_id")


In [52]:
df_train.fillna(0, inplace = True )
df_test.fillna(0, inplace = True )
df_competition.fillna(0, inplace = True)

In [53]:
df_train.columns

Index(['device_id', 'date', 'seconds_until_next_appearence', 'most_used_app',
       'os', 'Unnamed: 0_x', '1_Periodo_Installs', '2_Periodo_Installs',
       '3_Periodo_Installs', 'Cantidad_total_de_install', '1_Periodo_Auctions',
       '2_Periodo_Auctions', '3_Periodo_Auctions', 'Cantidad_total_de_auction',
       'Unnamed: 0_y', 'mejor_evento', 'cant_mejor_evento',
       'cant_total_de_eventos', 'Primer_Periodo', 'Segundo_Periodo',
       'Tercer_Periodo'],
      dtype='object')

In [54]:
df_train["mejor_evento"] = df_train["mejor_evento"].astype('category')

### Aca veo con que features me quedo

In [57]:
features = [#"device_id",
       'most_used_app', 'os','cant_total_de_eventos', 'mejor_evento', 'Primer_Periodo', 'Segundo_Periodo',
       'Tercer_Periodo']

In [71]:

#CREO X_TRAIN Y Y_TRAIN con DEVICE ID
X_train = df_train[features]
y_train = df_train['seconds_until_next_appearence']
X_test = df_test[features]
y_test = df_test['seconds_until_next_appearence']
X_competition = df_competition[['device_id',
       'most_used_app', 'os','cant_total_de_eventos', 'mejor_evento', 'Primer_Periodo', 'Segundo_Periodo',
       'Tercer_Periodo']]

#### Random Forest

In [59]:

from sklearn.ensemble import RandomForestRegressor

In [60]:
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler

In [241]:
from sklearn.model_selection import RandomizedSearchCV
# Number of features to consider at every split
# Create the random grid
random_grid = {'max_depth': range(3,7),
            'n_estimators': (10, 50, 100, 1000),
              }
print(random_grid)

{'max_depth': range(3, 7), 'n_estimators': (10, 50, 100, 1000)}


In [242]:
crf = RandomForestRegressor()
crf_random = RandomizedSearchCV(estimator= crf, param_distributions= random_grid,
                               n_iter = 100, cv = 3, verbose = 2, random_state=42, n_jobs=-1, scoring='neg_mean_squared_error')

In [ ]:
start_time = timer(None) # timing starts from this point for "start_time" variable
#crf_random.fit(X_train, y_train)
timer(start_time) # timing ends here for "start_time" variable

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:266: UserWarning: The total space of parameters 16 is smaller than n_iter=100. Running 16 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


In [ ]:
crf_random.best_params_

In [61]:
regr = RandomForestRegressor()
#entreno el modeo
regr.fit(X_train, y_train)  
#hago prediccion de X_test
y_pred = regr.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [62]:
from sklearn.metrics import mean_squared_error


In [63]:
#miro error X_test

print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))

Mean squared error: 148900798.94


In [67]:
#ahora predigo los ids que pide el TP
y_pred_competition = regr.predict(X_competition)

In [72]:
prediction_appearances = X_competition[["device_id"]]
prediction_appearances["seconds_until_next_appearance"] = y_pred_competition

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [73]:
prediction_appearances.head()

,device_id,seconds_until_next_appearance
0,1000169251625791246,14042.197077
1,1000395625957344683,2162.793205
2,1003027494996471685,9653.946055
3,1006670001679961544,4281.005844
4,1007573308966476713,23938.389733


In [74]:
#ahora hago la prediccion para tiempo en instalar

In [75]:
ids_to_predict_numbers_convertion = ids_to_predict_numbers_convertion.rename(columns={"ref_hash": "device_id"})

In [76]:
ids_to_predict_numbers_convertion.head()

,device_id,date
0,1000169251625791246,2019-05-27
2,1000395625957344683,2019-05-27
4,1003027494996471685,2019-05-27
6,1006670001679961544,2019-05-27
8,1007573308966476713,2019-05-27


In [77]:
#conversiones de los primeros 3 dias. uso para entrenar
convertions_train = pd.read_csv("auctions_deltas_to_convertion_train.csv",header=None, names=["device_id", "date", "seconds_until_convertion"])

In [78]:
#conversiones de los segunods 3 dias . uso para testear performance
convertions_test = pd.read_csv("auctions_deltas_to_convertion_test.csv",header=None, names=["device_id", "date", "seconds_until_convertion"])

In [79]:
convertions_train.head()

,device_id,date,seconds_until_convertion
0,8323952185725734415,2019-04-20 05:35:45.651502,259200.0
1,8323952185725734415,2019-04-20 05:40:46.744391,259200.0
2,8323952185725734415,2019-04-20 05:48:22.167361,259200.0
3,8323952185725734415,2019-04-20 05:51:43.749231,259200.0
4,8323952185725734415,2019-04-20 05:51:52.533312,259200.0


In [80]:
#agrego mismos features

In [81]:
df_train = convertions_train.merge(most_used_app, on="device_id", how="left")

In [82]:
df_test = convertions_test.merge(most_used_app, on="device_id", how="left")
df_competition = ids_to_predict_numbers_convertion.merge(most_used_app, on="device_id", how="left")

In [83]:
#renombro para dejar mas claro que el nombre del nuevo feature
df_train = df_train.rename({"application_id": "most_used_app"}, axis="columns")
df_test = df_test.rename({"application_id": "most_used_app"}, axis="columns")
df_competition = df_competition.rename({"application_id": "most_used_app"}, axis="columns")

In [84]:
#reemplazo los nan de most used app por un nuemero no existente
df_train["most_used_app"] = df_train["most_used_app"].fillna(-1)
df_test["most_used_app"] = df_test["most_used_app"].fillna(-1)
df_competition["most_used_app"] = df_competition["most_used_app"].fillna(-1)

In [85]:
#lo agrego a DF, siempre left join
df_train = df_train.merge(os,how="left", on="device_id")
df_test = df_test.merge(os,how="left", on="device_id")
df_competition = df_competition.merge(os,how="left", on="device_id")

In [86]:
df_train["os"] = df_train["os"].fillna("other")
df_test["os"] = df_test["os"].fillna("other")
df_competition["os"] = df_competition["os"].fillna("other")

In [87]:
#PRUEBO OTRA VEZ con random forest
#antes tengo que convertir a numeros los campos que son string (en este caso solo OS)
#USO LABEL ENCODER
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [88]:
#encodeo columna os
le.fit(df_train["os"])
df_train["os"] = le.transform(df_train["os"])
df_test["os"] = le.transform(df_test["os"])
df_competition["os"] = le.transform(df_competition["os"])

In [89]:
#lo agrego a DF, siempre left join
df_train = df_train.merge(features_cantidades,how="left", on="device_id")
df_test = df_test.merge(features_cantidades,how="left", on="device_id")
df_competition = df_competition.merge(features_cantidades,how="left", on="device_id")

In [90]:
df_train.fillna(0, inplace = True )
df_test.fillna(0, inplace = True )
df_competition.fillna(0, inplace = True)

In [91]:
df_train.columns

Index(['device_id', 'date', 'seconds_until_convertion', 'most_used_app', 'os',
       'Unnamed: 0_x', '1_Periodo_Installs', '2_Periodo_Installs',
       '3_Periodo_Installs', 'Cantidad_total_de_install', '1_Periodo_Auctions',
       '2_Periodo_Auctions', '3_Periodo_Auctions', 'Cantidad_total_de_auction',
       'Unnamed: 0_y', 'mejor_evento', 'cant_mejor_evento',
       'cant_total_de_eventos', 'Primer_Periodo', 'Segundo_Periodo',
       'Tercer_Periodo'],
      dtype='object')

#### Random Forest

In [92]:
features = [#"device_id",
       'most_used_app', 'os','cant_total_de_eventos', 'Cantidad_total_de_install', 'Cantidad_total_de_auction']

In [102]:

#CREO X_TRAIN Y Y_TRAIN con DEVICE ID
X_train = df_train[features]
y_train = df_train['seconds_until_convertion']
X_test = df_test[features]
y_test = df_test['seconds_until_convertion']
X_competition = df_competition[["device_id",
       'most_used_app', 'os','cant_total_de_eventos', 'Cantidad_total_de_install', 'Cantidad_total_de_auction']]

In [94]:
RandomForestRegressor()

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators='warn',
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [123]:
regr = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=2, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators='warn',
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)
#entreno el modeo
regr.fit(X_train, y_train)  
y_pred = regr.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [124]:
#miro error X_test

print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))

Mean squared error: 4648344712.02


In [100]:
#ahora predigo los ids que pide el TP
y_pred_competition = regr.predict(X_competition)

In [103]:
prediction_convertion = X_competition[["device_id"]]
prediction_convertion["seconds_until_convertion"] = y_pred_competition

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
#miro como me quedaron los dataframes de predicciones
#para submitear tiene que quedar con el formato que piden. o sea los ids de appearances tienen que 
#terminar en _st y los ids de convertions tienen que terminar en _sc

In [104]:
prediction_appearances.head()

,device_id,seconds_until_next_appearance
0,1000169251625791246,14042.197077
1,1000395625957344683,2162.793205
2,1003027494996471685,9653.946055
3,1006670001679961544,4281.005844
4,1007573308966476713,23938.389733


In [105]:
prediction_convertion.head()

,device_id,seconds_until_convertion
0,1000169251625791246,259200.000000
1,1000395625957344683,245053.319805
2,1003027494996471685,233245.289292
3,1006670001679961544,259200.000000
4,1007573308966476713,166407.062551


In [106]:
#paso al formato de submit

In [107]:
prediction_appearances = prediction_appearances.rename(columns={"device_id": "ref_hash", "seconds_until_next_appearance": "obj"})

In [108]:
prediction_convertion = prediction_convertion.rename(columns={"device_id": "ref_hash", "seconds_until_convertion": "obj"})

In [109]:
prediction_appearances["ref_hash"] = prediction_appearances["ref_hash"].apply(lambda ref: str(ref)+"_st")

In [110]:
prediction_convertion["ref_hash"] = prediction_convertion["ref_hash"].apply(lambda ref: str(ref)+"_sc")

In [111]:
prediction_appearances.head()

,ref_hash,obj
0,1000169251625791246_st,14042.197077
1,1000395625957344683_st,2162.793205
2,1003027494996471685_st,9653.946055
3,1006670001679961544_st,4281.005844
4,1007573308966476713_st,23938.389733


In [112]:
prediction_convertion.head()

,ref_hash,obj
0,1000169251625791246_sc,259200.000000
1,1000395625957344683_sc,245053.319805
2,1003027494996471685_sc,233245.289292
3,1006670001679961544_sc,259200.000000
4,1007573308966476713_sc,166407.062551


In [113]:
#concateno las predicciones porque tienen que devolverse todas juntas

In [114]:
full_pred = pd.concat([prediction_appearances, prediction_convertion])

In [115]:
#hago un left join con ids to predict para oredenarlos como pide el enunciado

In [116]:
ids_to_predict.head()

,ref_hash
0,1000169251625791246_sc
1,1000169251625791246_st
2,1000395625957344683_sc
3,1000395625957344683_st
4,1003027494996471685_sc


In [117]:
full_pred = ids_to_predict.merge(full_pred, on="ref_hash", how="left")

In [118]:
#ya esta igualito a como pide el enunciado
full_pred.head()

,ref_hash,obj
0,1000169251625791246_sc,259200.000000
1,1000169251625791246_st,14042.197077
2,1000395625957344683_sc,245053.319805
3,1000395625957344683_st,2162.793205
4,1003027494996471685_sc,233245.289292


In [119]:
#paso float de obj a int
full_pred["obj"] = full_pred["obj"].astype(np.int64)

In [120]:
full_pred.to_csv("first_pred.csv", index=False)

In [121]:
full_pred.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8074 entries, 0 to 8073
Data columns (total 2 columns):
ref_hash    8074 non-null object
obj         8074 non-null int64
dtypes: int64(1), object(1)
memory usage: 189.2+ KB


In [122]:
pd.read_csv("target_competencia_ids.csv").info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8074 entries, 0 to 8073
Data columns (total 2 columns):
ref_hash    8074 non-null object
obj         8074 non-null int64
dtypes: int64(1), object(1)
memory usage: 126.2+ KB
